In [218]:
import bs4 #beautifulsoup 4
from urllib.request import urlopen as uReq #url open module
from bs4 import BeautifulSoup as soup #beautifulsoup module
import datetime
import pandas as pd
import csv
import os
import string

In [219]:
#Baixar uma página
url = "https://store.steampowered.com/search/?specials=1&page=1"
uClient = uReq(url)
page_html = uClient.read()
uClient.close()

In [220]:
#Visualize o conteúdo dos dados no jogo
page_soup = soup(page_html, "html.parser")
containers = page_soup.findAll("div",{"class":"responsive_search_name_combined"})
print(len(containers))
print(containers[0])

25
<div class="responsive_search_name_combined">
<div class="col search_name ellipsis">
<span class="title">Red Dead Redemption 2</span>
<div>
<span class="platform_img win"></span> </div>
</div>
<div class="col search_released responsive_secondrow">5 Dec, 2019</div>
<div class="col search_reviewscore responsive_secondrow">
<span class="search_review_summary positive" data-tooltip-html="Very Positive&lt;br&gt;88% of the 247,002 user reviews for this game are positive.">
</span>
</div>
<div class="col search_price_discount_combined responsive_secondrow" data-price-final="11950">
<div class="col search_discount responsive_secondrow">
<span>-50%</span>
</div>
<div class="col search_price discounted responsive_secondrow">
<span style="color: #888888;"><strike>R$ 239,00</strike></span><br/>R$ 119,50                    </div>
</div>
</div>


In [221]:
#Inclua classificações positivas e o número de comentários, se houver
try:
    rating = containers[0].find("span", {"class","search_review_summary"})
    ratingtext = rating["data-tooltip-html"]
    splitrating = ratingtext.split(">")
    splitrating[1].split()
    percentile = splitrating[1].split()
    score = percentile[0]
    reviews = percentile[3]
    print(score, reviews)
except:
    print("No review data")

88% 247,002


In [222]:
#Faça um desconto
discount = containers[0].find("div", {"class","search_discount"})
print(discount.span.text)

-50%


In [223]:
#Obtenha os preços
price = containers[0].find("div", {"class","search_price"}).text
price2 = price.split("R$")
originalprice = price2[0].replace("\n","")
discountprice = price2[1]
print(originalprice, discountprice)

  239,00


In [224]:
#Obter datas
published = containers[1].find("div", {"class","search_released"})
print(published.text)

14 Jan, 2022


In [225]:
#Ele suporta janelas?
platformwin = containers[0].find("span", {"class","platform_img win"})
try:
    if len(platformwin) == 0:
        supportwin = 1
except TypeError:
    supportwin = 0
print(supportwin)

1


In [226]:
#Suporta mac?
platformmac = containers[0].find("span", {"class","platform_img mac"})
try:
    if len(platformmac) == 0:
        supportmac = 1
except TypeError:
    supportmac = 0
print(supportmac)

0


In [227]:
#Suporta linux?
platformlinux = containers[0].find("span", {"class","platform_img linux"})
try:
    if len(platformlinux) == 0:
        supportlinux = 1
except TypeError:
    supportlinux = 0
print(supportlinux)

0


In [228]:
#Data de hoje
now = datetime.datetime.now()
print (now.strftime("%Y-%m-%d %H:%M"))

2022-06-21 22:57


In [229]:
#Obter o título, converter para alterar e voltar para string (deve ser feito devido aos caracteres chineses, caso contrário não é possível escrever em csv)
title = containers[0].find("span", {"class","title"}).text
titlebyte = title.encode('ascii', 'ignore')
str(titlebyte, 'utf-8')

'Red Dead Redemption 2'

In [230]:
#Criar arquivo csv com cabeçalhos
filename="steamsales.csv"
file = open(filename, 'w')
writer = csv.writer(file)
if os.stat("steamsales.csv").st_size == 0:
    writer.writerow(["Title", "Rating", "reviews", "Discount", "Normal price", "discount price", "Publish date", "Win", "Mac", "Linux", "Date"])

In [231]:
#Os trechos de código anteriores adicionados em um loop
class Scraper:
    def scrape(self):
        for x in range(len(containers)):
            title = containers[x].find("span", {"class","title"}).text
            titlebyte = title.encode('ascii', 'ignore')
            title=str(titlebyte, 'utf-8')
            print(title)

            try:
                rating = containers[x].find("span", {"class","search_review_summary"})
                ratingtext = rating["data-tooltip-html"]
                splitrating = ratingtext.split(">")
                splitrating[1].split()
                percentile = splitrating[1].split()
                score = percentile[0]
                reviews = percentile[3].strip()
                print(score)
                print(reviews)
            except:
                print("No review data")
            discount = containers[x].find("div", {"class","search_discount"})
            price = containers[x].find("div", {"class","search_price"}).text
            price2 = price.split("R$")
            originalprice = price2[0].replace("\n","").strip()
            discountprice = price2[1].strip()
            published = containers[x].find("div", {"class","search_released"}).text
            discounted = discount.text.strip()
            print(discounted)
            print(originalprice)
            print(discountprice)
            print(published)
            platformwin = containers[x].find("span", {"class","platform_img win"})
            try:
                if len(platformwin) == 0:
                    supportwin = 1
            except TypeError:
                supportwin = 0
            print("Windows support:",supportwin)

            platformmac = containers[x].find("span", {"class","platform_img mac"})
            try:
                if len(platformmac) == 0:
                    supportmac = 1
            except TypeError:
                supportmac = 0
            print("Mac support:",supportmac)

            platformlinux = containers[x].find("span", {"class","platform_img linux"})
            try:
                if len(platformlinux) == 0:
                    supportlinux = 1
            except TypeError:
                supportlinux = 0
            print("Linux support:",supportlinux)
            now = datetime.datetime.now()
            print (now.strftime("%Y-%m-%d %H:%M"))
            saledate= now.strftime("%Y-%m-%d %H:%M")
            writer.writerow([title,score,reviews,discounted,originalprice,discountprice, published,supportwin,supportmac,supportlinux,saledate])
scraper = Scraper()
scraper.scrape()

Red Dead Redemption 2
88%
247,002
-50%

239,00
5 Dec, 2019
Windows support: 1
Mac support: 0
Linux support: 0
2022-06-21 22:57
God of War
97%
34,532
-20%

199,90
14 Jan, 2022
Windows support: 1
Mac support: 0
Linux support: 0
2022-06-21 22:57
DayZ
71%
255,764
-40%

119,99
13 Dec, 2018
Windows support: 1
Mac support: 0
Linux support: 0
2022-06-21 22:57
FINAL FANTASY VII REMAKE INTERGRADE
91%
2,070
-29%

349,90
17 Jun, 2022
Windows support: 1
Mac support: 0
Linux support: 0
2022-06-21 22:57
Raft
93%
151,608
-15%

36,99
20 Jun, 2022
Windows support: 1
Mac support: 0
Linux support: 0
2022-06-21 22:57
MONSTER HUNTER RISE
87%
21,593
-49%

179,99
12 Jan, 2022
Windows support: 1
Mac support: 0
Linux support: 0
2022-06-21 22:57
A Plague Tale: Innocence
93%
37,791
-80%

129,90
14 May, 2019
Windows support: 1
Mac support: 0
Linux support: 0
2022-06-21 22:57
DayZ Livonia Edition
71%
256,720
-43%

144,98

Windows support: 1
Mac support: 0
Linux support: 0
2022-06-21 22:57
Dying Light 2 Stay Human
7

In [232]:
#Baixe páginas e armazene os dados em um csv
for x in range(0):
    page=str(x+1)
    url = ("https://store.steampowered.com/search/?specials=1&page="+page)
    uClient = uReq(url)
    page_html = uClient.read()
    page_soup = soup(page_html, "html.parser")
    containers = page_soup.findAll("div",{"class":"responsive_search_name_combined"})
    scraper.scrape()
    uClient.close()

In [233]:
file.close()